### Loading pretrained model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
import json
# for debug mode
sys.path.insert(0, "/opt/math/jupyter/ntr/Experiments_with_text")

In [2]:
from fairapi.modules.sentence_bert_model import SentenceBertModel
from fairapi.modules.logging_utils import *
logger = get_logger(name=ROOT_LOGGER_NAME,
                    console=True,
                    log_level="INFO",
                    propagate=False)

In [15]:
config = {'bert_model_path': '/opt/math/jupyter/ntr/Experiments_with_text/models_filtered/bert_fine/bert_model_trained', 
          'classification_model_path': '/opt/math/jupyter/ntr/Experiments_with_text/models_filtered/bert_fine/classifier_model', 
          'cache_path': ''}

In [16]:
model = SentenceBertModel(logger, **config)

[2021-05-09 10:11:04][NLI_API][INFO] Models are loaded and ready to use.
[2021-05-09 10:11:04][NLI_API][INFO] Loading cache...
[2021-05-09 10:11:04][NLI_API][INFO] Cache is not used.


### Transfer learning validation

## FEVER FILLED NEI CLASS

In [17]:
import pandas as pd
test_fever = pd.read_csv('../../data/FEVER/test_oversampled.csv')
test_fever.dropna(inplace = True)
test_fever.drop_duplicates(inplace = True)
test_fever.reset_index(drop=True, inplace=True)
test_fever.head()

,claim,hypothesis,label,query,candidates
0,Colin Kaepernick became a starting quarterback...,\tThese quarterbacks have started at least one...,NOT ENOUGH INFO,Colin Kaepernick became a starting quarterback...,{'List_of_San_Francisco_49ers_starting_quarter...
1,Tilda Swinton is a vegan.,\tShe next starred in Sally Potter 's Orlando ...,NOT ENOUGH INFO,Tilda Swinton is a vegan.,"{'Tilda_Swinton', 'John_Swinton_of_Kimmerghame..."
2,Tilda Swinton is a vegan.,"\tIn 2005 , Swinton was given the Richard Harr...",NOT ENOUGH INFO,Tilda Swinton is a vegan.,"{'Tilda_Swinton', 'John_Swinton_of_Kimmerghame..."
3,Tilda Swinton is a vegan.,\tShe followed this with appearances in Vanill...,NOT ENOUGH INFO,Tilda Swinton is a vegan.,"{'Tilda_Swinton', 'John_Swinton_of_Kimmerghame..."
4,Tilda Swinton is a vegan.,"\tShe is the recipient of two BAFTA Awards , o...",NOT ENOUGH INFO,Tilda Swinton is a vegan.,"{'Tilda_Swinton', 'John_Swinton_of_Kimmerghame..."


In [18]:
from tqdm.auto import tqdm
preds = model.predict_batch(test_fever.claim, test_fever.hypothesis)

In [19]:
preds_df_fever = pd.DataFrame(preds)

preds_df_fever['label_true'] = test_fever['label']
preds_df_fever['label_true'] = preds_df_fever['label_true']

import numpy as np
np.mean(preds_df_fever.label == preds_df_fever.label_true)

0.7348478099480327

In [20]:
preds_df_fever

,label,contradiction_prob,entailment_prob,neutral_prob,label_true
0,NOT ENOUGH INFO,0.000000,0.460723,0.539277,NOT ENOUGH INFO
1,NOT ENOUGH INFO,0.375752,0.000000,0.624248,NOT ENOUGH INFO
2,NOT ENOUGH INFO,0.414830,0.000000,0.585170,NOT ENOUGH INFO
3,NOT ENOUGH INFO,0.376625,0.000000,0.623375,NOT ENOUGH INFO
4,NOT ENOUGH INFO,0.399986,0.000000,0.600014,NOT ENOUGH INFO
...,...,...,...,...,...
33670,NOT ENOUGH INFO,0.000000,0.085439,0.914561,NOT ENOUGH INFO
33671,NOT ENOUGH INFO,0.000000,0.108806,0.891194,NOT ENOUGH INFO
33672,REFUTES,0.884510,0.000000,0.115490,REFUTES
33673,REFUTES,0.542505,0.000000,0.457495,REFUTES


In [21]:
preds_df_fever_filtered = preds_df_fever[preds_df_fever['label_true'].isin(['SUPPORTS', 'REFUTES'])]
np.mean(preds_df_fever_filtered.label == preds_df_fever_filtered.label_true) 

0.711097928265036

In [22]:
# Importing the dependancies
from sklearn import metrics
# Predicted values
y_pred = preds_df_fever['label']
# Actual values
y_act = preds_df_fever['label_true']
# Printing the confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
print(metrics.confusion_matrix(y_act, y_pred, labels=['SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO']))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=['SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO']))

[[8628  677 1476]
 [2295 6955 1883]
 [1563 1035 9163]]
                 precision    recall  f1-score   support

       SUPPORTS       0.69      0.80      0.74     10781
        REFUTES       0.80      0.62      0.70     11133
NOT ENOUGH INFO       0.73      0.78      0.75     11761

       accuracy                           0.73     33675
      macro avg       0.74      0.73      0.73     33675
   weighted avg       0.74      0.73      0.73     33675



## Validating on cleaned hypothesis

In [23]:
def clean_hyp(text):
    text = str(text)
    dot_id = text.rfind('.')
    if dot_id > 0:
        text = text[1:dot_id+1]
    else:
        text = text[1:]
        t_id = text.find('\t')
        if t_id > 0:
            text = text[:t_id]
    return text

test_fever.hypothesis = test_fever.hypothesis.apply(clean_hyp)

from tqdm.auto import tqdm
preds = model.predict_batch(test_fever.claim, test_fever.hypothesis)

In [24]:
preds_df_fever = pd.DataFrame(preds)

preds_df_fever['label_true'] = test_fever['label']
preds_df_fever['label_true'] = preds_df_fever['label_true']

import numpy as np
np.mean(preds_df_fever.label == preds_df_fever.label_true)

0.7338678544914625

In [25]:
preds_df_fever_filtered = preds_df_fever[preds_df_fever['label_true'].isin(['SUPPORTS', 'REFUTES'])]
np.mean(preds_df_fever_filtered.label == preds_df_fever_filtered.label_true) 

0.7044355206717167

In [26]:
# Importing the dependancies
from sklearn import metrics
# Predicted values
y_pred = preds_df_fever['label']
# Actual values
y_act = preds_df_fever['label_true']
# Printing the confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
print(metrics.confusion_matrix(y_act, y_pred, labels=['SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO']))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=['SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO']))

[[8620  612 1549]
 [2359 6817 1957]
 [1528  957 9276]]
                 precision    recall  f1-score   support

       SUPPORTS       0.69      0.80      0.74     10781
        REFUTES       0.81      0.61      0.70     11133
NOT ENOUGH INFO       0.73      0.79      0.76     11761

       accuracy                           0.73     33675
      macro avg       0.74      0.73      0.73     33675
   weighted avg       0.74      0.73      0.73     33675



## Model performance depending on length of text

In [43]:
correctness = (preds_df_fever.label == preds_df_fever.label_true).values
length = test_fever.hypothesis.apply(lambda x: len(x.split(' ')))

df = pd.DataFrame({'correctness': correctness, 'length':length})

one_per = len(df)*0.01

In [44]:
import math 

stats = df.groupby('length').agg({'correctness': ['mean', 'count', 'std'], 'length':['first']})
stats = stats[stats['correctness']['count'] > one_per]

print(stats)
print('-'*30)

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s, _ = stats.loc[i]
    ci95_hi.append(m + 1.95*s/math.sqrt(c))
    ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
stats.columns = list(stats.columns.droplevel())[:-2] + ['ci95_hi', 'ci95_lo']
print(stats)

       correctness                 length
              mean count       std  first
length                                   
10        0.743487   499  0.437146     10
11        0.771255   494  0.420450     11
12        0.737374   594  0.440432     12
13        0.823444   691  0.381568     13
14        0.736905   840  0.440576     14
15        0.745506  1057  0.435783     15
16        0.738289   982  0.439790     16
17        0.762800   957  0.425588     17
18        0.761155  1143  0.426565     18
19        0.787743  1338  0.409059     19
20        0.775168  1192  0.417647     20
21        0.773946  1305  0.418435     21
22        0.739408  1251  0.439133     22
23        0.768519  1296  0.421942     23
24        0.743750  1280  0.436732     24
25        0.753690  1084  0.431060     25
26        0.743205  1067  0.437070     26
27        0.761905   987  0.426134     27
28        0.771292  1045  0.420202     28
29        0.746835  1185  0.435008     29
30        0.738232   871  0.439850

In [45]:
import altair as alt
import pandas as pd
import numpy as np

# the base chart
base = alt.Chart(stats)

# generate the points
points = base.mark_point(
    filled=True,
    size=50,
    color='black'
).encode(
    x=alt.X('first:Q', title = 'Accuracy', scale=alt.Scale(domain=(5, 45))),
    y=alt.Y('mean:Q', title = 'Length of sentence', scale=alt.Scale(domain=(0.6, 0.9)))
)

# generate the error bars
errorbars = base.mark_errorbar().encode(
    x=alt.X("first:Q", title = ''),
    y=alt.Y("ci95_lo:Q",title = ''),
    y2="ci95_hi:Q"
)

(points + errorbars).properties(
    width=600,
    height=300
)

alt.LayerChart(...)